# SageMaker Inference

The purpose of this notebook is to show how API Gateway and Lambda can be setup to make an inference on a SageMaker endpoint which is hosting an XGBoost model. 

### Prerequisite
This notebook requires **Lab 2** from the SageMaker Immersion day has been completed and the SageMaker endpoint is running.

In [1]:
import boto3
import pandas as pd
import numpy as np

smr = boto3.client('sagemaker-runtime')

In [2]:
import sagemaker
#sess = sagemaker.Session(boto3.session.Session(region_name='us-west-2'))
sess = sagemaker.Session(boto3.session.Session())

In [3]:
role = 'arn:aws:iam::364430515305:role/service-role/AmazonSageMaker-ExecutionRole-20210301T212321'

bucket=sess.default_bucket()
prefix = 'macbookpro/learning-sagemaker'

test_path = f"s3://{bucket}/{prefix}/test"

!aws s3 cp $test_path/test_x.csv /tmp/test_x.csv
!aws s3 cp $test_path/test_y.csv /tmp/test_y.csv

download: s3://sagemaker-us-west-2-364430515305/macbookpro/learning-sagemaker/test/test_x.csv to ../../../../../tmp/test_x.csv
download: s3://sagemaker-us-west-2-364430515305/macbookpro/learning-sagemaker/test/test_y.csv to ../../../../../tmp/test_y.csv


In [4]:
X_test = pd.read_csv('/tmp/test_x.csv', names=[f'{i}' for i in range(59)])
y_test = pd.read_csv('/tmp/test_y.csv', names=['y'])

pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)  

X_test.drop(X_test.columns[0], axis=1, inplace=True)

In [5]:
X_test.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58
0,3,999,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,1,999,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
2,1,999,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0
3,3,999,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
4,1,999,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [6]:
sm_client = boto3.client('sagemaker')
endpoint = sm_client.list_endpoints(SortBy='CreationTime')['Endpoints'][0]['EndpointName']
print(f'endpoint name: {endpoint}')

def make_prediction(X_test, y_test, row, endpoint=endpoint):
    
    # get individual row from dataframe and format as csv string
    sample = X_test.iloc[row].values
    sample = np.array2string(sample, separator=',')
    sample = sample.strip('[').strip(']').lstrip(' ')
    sample = sample.replace('\n', '')
    sample.encode('utf-8')
    
    # create sagemaker runtime client and invoke sagemaker endpoint
    smr = boto3.client('sagemaker-runtime')
    r = smr.invoke_endpoint(EndpointName=endpoint, Body=sample, ContentType='text/csv')
    prediction = r['Body'].read().decode('utf-8')
    prediction = float(prediction)
    
    # round to 4 decimal places
    prediction = round(prediction, 4)
    return prediction

endpoint name: xgboost-210924-0315-016-aaedc4f9


Make a prediction againsts the SageMaker endpoint

In [7]:
make_prediction(X_test, y_test, 260)

0.7026

The label for sample 260 is a positive prediction that the customer will purchase a CD investment.

In [9]:
y_test.iloc[260].values

array([1])

## Lambda Setup

In [9]:
%%writefile sagemaker_inference_function.py

import json
import boto3

smr = boto3.client('sagemaker-runtime')

def lambda_handler(event, context):
    
    # check event type and print event
    print(f'event data type: {type(event)}')
    print(event)
    
    # grab body and convert to dict
    body = json.loads(event['body'])
    
    # encode the sample to bytes
    data = body['data'].encode('utf-8')
    
    # check data types
    print(f'body data type: {type(body)}')
    print(body)

    print(f'data data type: {type(data)}')
    print(data)

    # make prediction on sagemaker endpoint
    r = smr.invoke_endpoint(EndpointName='xgboost-210924-0315-016-aaedc4f9', Body=data, ContentType='text/csv')
    prediction = r['Body'].read().decode('utf-8')
    print(prediction)
    response = {'prediction': prediction}
    
    # return prediction in body
    return {
        'statusCode': 200,
        'body': json.dumps(response)
    }


Overwriting sagemaker_inference_function.py


In [10]:
!zip sm-inference-package.zip sagemaker_inference_function.py

updating: sagemaker_inference_function.py (deflated 52%)


In [11]:
lambda_client = boto3.client('lambda')

ZIPNAME = "sm-inference-package.zip"


def aws_file():
    with open(ZIPNAME, 'rb') as file_data:
        bytes_content = file_data.read()
    return bytes_content


FunctionName = 'sagemaker-inference-xgboost'

response = lambda_client.create_function(Code={'ZipFile': aws_file()},
                                         Description='Hello World Test.',
                                         FunctionName=FunctionName,
                                         Handler='sagemaker_inference_function.lambda_handler',
                                         Publish=True,
                                         Role='arn:aws:iam::364430515305:role/service-role/xgboost_sagemaker_inference-role-24efeipb',
                                         Runtime='python3.9')
response

ResourceConflictException: An error occurred (ResourceConflictException) when calling the CreateFunction operation: Function already exist: sagemaker-inference-xgboost

In [11]:
IntegrationUri = response['FunctionArn']

In [12]:
IntegrationUri

'arn:aws:lambda:us-west-2:364430515305:function:sagemaker-inference-xgboost'

In [ ]:
#IntegrationUri = response['FunctionName']
#FunctionName = response['FunctionName']

## Lambda test event

Setup a test event by copying the following json into a lambda test:

```json
{
  "version": "2.0",
  "routeKey": "POST /predict",
  "rawPath": "/predict",
  "rawQueryString": "",
  "headers": {
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br",
    "cache-control": "no-cache",
    "content-length": "127",
    "content-type": "text/csv",
    "host": "8fpcmkyo88.execute-api.us-west-2.amazonaws.com",
    "postman-token": "2c03b0e2-3f21-434b-879d-c7a7d7ff7633",
    "user-agent": "PostmanRuntime/7.28.4",
    "x-amzn-trace-id": "Root=1-614e5cb1-59e2e394227dfc3213740435",
    "x-forwarded-for": "54.86.50.139",
    "x-forwarded-port": "443",
    "x-forwarded-proto": "https"
  },
  "requestContext": {
    "accountId": "364430515305",
    "apiId": "8fpcmkyo88",
    "domainName": "8fpcmkyo88.execute-api.us-west-2.amazonaws.com",
    "domainPrefix": "8fpcmkyo88",
    "http": {
      "method": "POST",
      "path": "/predict",
      "protocol": "HTTP/1.1",
      "sourceIp": "54.86.50.139",
      "userAgent": "PostmanRuntime/7.28.4"
    },
    "requestId": "GMNryjUfvHcEMjA=",
    "routeKey": "POST /predict",
    "stage": "$default",
    "time": "24/Sep/2021:23:18:09 +0000",
    "timeEpoch": 1632525489647
  },
  "body": "{\"data\": \"3,3,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1\"}",
  "isBase64Encoded": false
}
```

# Create API Gateway

In [13]:
def create_apigateway(Name='sagemaker-inference-xgboost', IntegrationUri=IntegrationUri):
    api_client = boto3.client('apigatewayv2')
    response = api_client.create_api(Name=Name, ProtocolType='HTTP')
    ApiEndpoint = response['ApiEndpoint']
    ApiId = response['ApiId']
    response = api_client.create_integration(ApiId=ApiId, 
                                         IntegrationType='AWS_PROXY', 
                                         IntegrationUri=IntegrationUri,
                                         PayloadFormatVersion='2.0')
    Target = response['IntegrationId']
    IntegrationId = response['IntegrationId']
    response = api_client.create_route(ApiId=ApiId, RouteKey='POST /predict', Target='integrations/'+Target)
    RouteKey = response['RouteKey'].split()[1]
    response = api_client.create_stage(ApiId=ApiId, StageName='v1')
    StageName = response['StageName']
    response = api_client.get_integration(ApiId=ApiId, IntegrationId=IntegrationId)
    SourceArn = 'arn:aws:execute-api:us-west-2:364430515305:' + ApiId + '/*/*/predict'
    lambda_client = boto3.client('lambda')
    response = lambda_client.add_permission(FunctionName=FunctionName, 
                                            StatementId='1',
                                            Action='lambda:InvokeFunction',
                                            Principal='apigateway.amazonaws.com',
                                            SourceArn=SourceArn)
    response = api_client.create_deployment(ApiId=ApiId, StageName=StageName)
    url = ApiEndpoint + '/' + StageName + RouteKey
    return url

In [14]:
url = create_apigateway()
url

'https://n2lxrl5pni.execute-api.us-west-2.amazonaws.com/v1/predict'

In [15]:
# Helper function to get a single row from test dataframe
def get_sample(X_test, row):
    sample = X_test.iloc[row].values
    sample = np.array2string(sample, separator=',')
    sample = sample.strip('[').strip(']').lstrip(' ')
    sample = sample.replace('\n', '')
    sample = sample.replace(' ', '')
    return sample

## Make Inference with POST request to API Gateway and Lambda

In [18]:
import requests

payload = {'data': get_sample(X_test, 260)}
print(payload)

r = requests.post(url, json=payload)
r.text

{'data': '3,3,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1'}


'{"prediction": "0.7025916576385498"}'

## Flask

In [22]:
#%pip install flask

In [24]:
# from flask import Flask
# from flask import request, jsonify
# import boto3

# app = Flask(__name__)

# smr = boto3.client('sagemaker-runtime')

# @app.route('/pred', methods = ['POST'])
# def predict():
#     # make prediction on sagemaker endpoint
#     body = request.get_json()
#     print(type(body))
#     print(body)
#     data = body['data']
#     r = smr.invoke_endpoint(EndpointName='xgboost-210924-0315-016-aaedc4f9', Body=data, ContentType='text/csv')
#     prediction = r['Body'].read().decode('utf-8')
#     print(prediction)
#     response = {'prediction': prediction}
#     return jsonify(response)

# app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [ ]:
# curl --location --request POST 'http://localhost:5000/pred' \
#                          --header 'Content-Type: application/json' \
#                          --data-raw '{
#                          "data": "3,3,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1"
#                      }'

#### Launch EC2 instance

In [30]:
ec2_client = boto3.client('ec2')

#### Need to create security group to allow port 5000

In [60]:
user_data = '''#!/bin/bash
echo 'test' > /tmp/hello
yum update -y
yum install python3 python3-pip tmux htop -y
pip3 install flask boto3
wget https://raw.githubusercontent.com/sciarrilli/flask-sagemaker-inference/main/flask-app.py

tmux new-session -d -s flask-session
tmux send-keys 'python3 flask-app.py' C-m
tmux detach -s flask-session'''

In [62]:
instances = ec2_client.run_instances(
    ImageId="ami-0c2d06d50ce30b442",
    MinCount=1,
    MaxCount=1,
    InstanceType="t3.small",
    KeyName='macos_2020',
    UserData=user_data,
    IamInstanceProfile={
        'Name': 'EC2-SageMaker-Inference'
    },
    TagSpecifications=[
    {
        'ResourceType': 'instance',
        'Tags': [
            {
                'Key': 'Name',
                'Value': 'flask-sagemaker-inference'
            },
        ]
    },
])

InstanceId = instances['Instances'][0]['InstanceId']



In [67]:
response = ec2_client.describe_instances(
    InstanceIds=[
        InstanceId,
    ]
)

In [68]:
response['Reservations'][0]['Instances'][0]['PublicIpAddress']

'18.237.13.241'

#### Need to add IP address as a bash variable for curl command

In [72]:
%%bash
curl -s --location --request POST 'http://18.237.13.241:5000/pred' \
--header 'Content-Type: application/json' \
--data-raw '{"data": "3,3,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1"}'

{
  "prediction": "0.7025916576385498"
}


# Appendix

## API Gateway Setup

In [106]:
api_client = boto3.client('apigatewayv2')

In [107]:
response = api_client.create_api(Name='sagemaker-apigw', ProtocolType='HTTP')
response

{'ResponseMetadata': {'RequestId': '3cbfa6ce-b52c-406f-b9b8-440b4d0cbfe9',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sat, 25 Sep 2021 01:09:23 GMT',
   'content-type': 'application/json',
   'content-length': '326',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3cbfa6ce-b52c-406f-b9b8-440b4d0cbfe9',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'GMd-mF1zvHcFg5Q=',
   'access-control-expose-headers': 'x-amzn-RequestId,x-amzn-ErrorType,x-amzn-ErrorMessage,Date',
   'x-amzn-trace-id': 'Root=1-614e76c3-656eae703fa4882858738a51'},
  'RetryAttempts': 0},
 'ApiEndpoint': 'https://6raxaoa7bd.execute-api.us-west-2.amazonaws.com',
 'ApiId': '6raxaoa7bd',
 'ApiKeySelectionExpression': '$request.header.x-api-key',
 'CreatedDate': datetime.datetime(2021, 9, 25, 1, 9, 23, tzinfo=tzlocal()),
 'DisableExecuteApiEndpoint': False,
 'Name': 'sagemaker-apigw',
 'ProtocolType': 'HTTP',
 'RouteSelectionExpression': '$request.method $request.path'}

In [108]:
ApiEndpoint = response['ApiEndpoint']
ApiId = response['ApiId']

In [109]:
#
#
#
#
#
# NEED TO COMMENT OUT
#
#
#
#
IntegrationUri = 'arn:aws:lambda:us-west-2:364430515305:function:sagemaker-inference-test'

In [110]:
response = api_client.create_integration(ApiId=ApiId, 
                                         IntegrationType='AWS_PROXY', 
                                         IntegrationUri=IntegrationUri,
                                         PayloadFormatVersion='2.0')
response

{'ResponseMetadata': {'RequestId': '1c10820a-d605-4a40-8d53-b20f530e2645',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sat, 25 Sep 2021 01:09:30 GMT',
   'content-type': 'application/json',
   'content-length': '256',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1c10820a-d605-4a40-8d53-b20f530e2645',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'GMd_nGr0vHcFowg=',
   'access-control-expose-headers': 'x-amzn-RequestId,x-amzn-ErrorType,x-amzn-ErrorMessage,Date',
   'x-amzn-trace-id': 'Root=1-614e76ca-7af1a157696e798d310db7bc'},
  'RetryAttempts': 0},
 'ConnectionType': 'INTERNET',
 'IntegrationId': '2mc2y17',
 'IntegrationMethod': 'POST',
 'IntegrationType': 'AWS_PROXY',
 'IntegrationUri': 'arn:aws:lambda:us-west-2:364430515305:function:sagemaker-inference-test',
 'PayloadFormatVersion': '2.0',
 'TimeoutInMillis': 30000}

In [111]:
Target = response['IntegrationId']
IntegrationId = response['IntegrationId']

In [112]:
response = api_client.create_route(ApiId=ApiId, RouteKey='POST /predict', Target='integrations/'+Target)
response

{'ResponseMetadata': {'RequestId': '338d1a0d-f599-4ec1-97d1-e03d79ec88f4',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sat, 25 Sep 2021 01:09:34 GMT',
   'content-type': 'application/json',
   'content-length': '130',
   'connection': 'keep-alive',
   'x-amzn-requestid': '338d1a0d-f599-4ec1-97d1-e03d79ec88f4',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'GMeAQGYRPHcF7qg=',
   'access-control-expose-headers': 'x-amzn-RequestId,x-amzn-ErrorType,x-amzn-ErrorMessage,Date',
   'x-amzn-trace-id': 'Root=1-614e76ce-095e31e2409cc6207167b29c'},
  'RetryAttempts': 0},
 'ApiKeyRequired': False,
 'AuthorizationType': 'NONE',
 'RouteId': 'yl6n1t5',
 'RouteKey': 'POST /predict',
 'Target': 'integrations/2mc2y17'}

In [113]:
RouteKey = response['RouteKey'].split()[1]

In [114]:
response = api_client.create_stage(ApiId=ApiId, StageName='v1')
response

{'ResponseMetadata': {'RequestId': '5f2e5e45-ca1e-4b17-87a4-ef7ea4508774',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sat, 25 Sep 2021 01:09:38 GMT',
   'content-type': 'application/json',
   'content-length': '250',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5f2e5e45-ca1e-4b17-87a4-ef7ea4508774',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'GMeA7GZEPHcF-EA=',
   'access-control-expose-headers': 'x-amzn-RequestId,x-amzn-ErrorType,x-amzn-ErrorMessage,Date',
   'x-amzn-trace-id': 'Root=1-614e76d2-0f132a7f10e0f5cf33d8934c'},
  'RetryAttempts': 0},
 'CreatedDate': datetime.datetime(2021, 9, 25, 1, 9, 38, tzinfo=tzlocal()),
 'DefaultRouteSettings': {'DetailedMetricsEnabled': False},
 'LastUpdatedDate': datetime.datetime(2021, 9, 25, 1, 9, 38, tzinfo=tzlocal()),
 'RouteSettings': {},
 'StageName': 'v1',
 'StageVariables': {},
 'Tags': {}}

In [115]:
#ApiEndpoint = 'https://fuyk7jxq9e.execute-api.us-west-2.amazonaws.com'
StageName = response['StageName']
#RouteKey = '/predict'

In [119]:
response = api_client.get_integration(ApiId=ApiId, IntegrationId=IntegrationId)
response

{'ResponseMetadata': {'RequestId': '4f747534-263b-43fe-bdb4-b8cd5f875fd4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 25 Sep 2021 01:09:46 GMT',
   'content-type': 'application/json',
   'content-length': '256',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4f747534-263b-43fe-bdb4-b8cd5f875fd4',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'GMeCLGpjPHcF6vg=',
   'access-control-expose-headers': 'x-amzn-RequestId,x-amzn-ErrorType,x-amzn-ErrorMessage,Date',
   'x-amzn-trace-id': 'Root=1-614e76da-1951ccb33ccd17926aaa3784'},
  'RetryAttempts': 0},
 'ConnectionType': 'INTERNET',
 'IntegrationId': '2mc2y17',
 'IntegrationMethod': 'POST',
 'IntegrationType': 'AWS_PROXY',
 'IntegrationUri': 'arn:aws:lambda:us-west-2:364430515305:function:sagemaker-inference-test',
 'PayloadFormatVersion': '2.0',
 'TimeoutInMillis': 30000}

In [120]:
ApiId

'6raxaoa7bd'

In [121]:
SourceArn = 'arn:aws:execute-api:us-west-2:364430515305:' + ApiId + '/*/*/predict'

In [122]:
SourceArn

'arn:aws:execute-api:us-west-2:364430515305:6raxaoa7bd/*/*/predict'

In [125]:
#
#
#
#
#  Delete
#
#
#
#
FunctionName = 'sagemaker-inference-test'

In [126]:
lambda_client = boto3.client('lambda')

response = lambda_client.add_permission(FunctionName=FunctionName, 
                                        StatementId='1',
                                        Action='lambda:InvokeFunction',
                                        Principal='apigateway.amazonaws.com',
                                        SourceArn=SourceArn)

In [127]:
response

{'ResponseMetadata': {'RequestId': '8810eb9d-05f5-4383-9ef3-40e22977405c',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sat, 25 Sep 2021 01:22:25 GMT',
   'content-type': 'application/json',
   'content-length': '353',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8810eb9d-05f5-4383-9ef3-40e22977405c'},
  'RetryAttempts': 0},
 'Statement': '{"Sid":"1","Effect":"Allow","Principal":{"Service":"apigateway.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-west-2:364430515305:function:sagemaker-inference-test","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:execute-api:us-west-2:364430515305:6raxaoa7bd/*/*/predict"}}}'}

In [131]:
response = api_client.create_deployment(ApiId=ApiId, StageName=StageName)
response

{'ResponseMetadata': {'RequestId': '8c9514b0-0120-491f-bd6a-61f9ae9fb578',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Sat, 25 Sep 2021 01:24:44 GMT',
   'content-type': 'application/json',
   'content-length': '113',
   'connection': 'keep-alive',
   'x-amzn-requestid': '8c9514b0-0120-491f-bd6a-61f9ae9fb578',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'GMgOWFJXPHcFsiA=',
   'access-control-expose-headers': 'x-amzn-RequestId,x-amzn-ErrorType,x-amzn-ErrorMessage,Date',
   'x-amzn-trace-id': 'Root=1-614e7a5b-6ca9e6250b7b8dd75a5a81f8'},
  'RetryAttempts': 0},
 'AutoDeployed': False,
 'CreatedDate': datetime.datetime(2021, 9, 25, 1, 24, 43, tzinfo=tzlocal()),
 'DeploymentId': 'ibs3gb',
 'DeploymentStatus': 'DEPLOYED'}

In [105]:
# %%bash
# aws lambda add-permission \
#   --statement-id 03e827d2-c823-5826-83c9-3ca0380917e8 \
#   --action lambda:InvokeFunction \
#   --function-name "arn:aws:lambda:us-west-2:364430515305:function:sagemaker-inference-test" \
#   --principal apigateway.amazonaws.com \
#   --source-arn "arn:aws:execute-api:us-west-2:364430515305:djyoy2gtbj/*/*/predict"

{
    "Statement": "{\"Sid\":\"03e827d2-c823-5826-83c9-3ca0380917e8\",\"Effect\":\"Allow\",\"Principal\":{\"Service\":\"apigateway.amazonaws.com\"},\"Action\":\"lambda:InvokeFunction\",\"Resource\":\"arn:aws:lambda:us-west-2:364430515305:function:sagemaker-inference-test\",\"Condition\":{\"ArnLike\":{\"AWS:SourceArn\":\"arn:aws:execute-api:us-west-2:364430515305:djyoy2gtbj/*/*/predict\"}}}"
}


In [128]:
print(ApiEndpoint, StageName, RouteKey)

https://6raxaoa7bd.execute-api.us-west-2.amazonaws.com v1 /predict


In [129]:
url = ApiEndpoint + '/' + StageName + RouteKey

In [130]:
url

'https://6raxaoa7bd.execute-api.us-west-2.amazonaws.com/v1/predict'